# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.


In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [37]:
city_data = pd.read_csv("../output_data/cities.csv")
city_data

,Unnamed: 0,City,Latitude,Longitude,Temperature,Cloudiness,Humidity,Wind,Country,Date
0,0,Sweetwater,25.7634,-80.3731,78.01,75,60,6.91,US,1616264529
1,1,Codrington,-38.2667,141.9667,58.75,100,90,7.52,AU,1616264529
2,2,Ushuaia,-54.8000,-68.3000,48.20,40,57,26.46,AR,1616264530
3,3,New Norfolk,-42.7826,147.0587,60.80,90,59,5.75,AU,1616263936
4,4,Vaini,-21.2000,-175.2000,75.20,40,100,3.44,TO,1616263949
...,...,...,...,...,...,...,...,...,...,...
550,550,Nabīnagar,23.8917,90.9733,82.40,40,61,7.00,BD,1616264737
551,551,Buin,-33.7333,-70.7500,73.40,20,31,4.61,CL,1616264737
552,552,Surgut,61.2500,73.4167,23.00,75,100,4.47,RU,1616264738
553,553,Ziyang,29.2497,117.8547,50.36,100,87,3.91,CN,1616264738


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
#configure 
gmaps.configure(api_key = g_key)

locations = city_data[["Latitude", "Longitude"]]
weight = city_data["Humidity"]

In [39]:
#heatmap 
figure = gmaps.figure(center=(46.0, -5.0), zoom_level = 1.7)
max_intensity = np.max(weight)

#heat layer
heat_map_layer = gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity =100, point_radius=3)

#add heat layer
figure.add_layer(heat_map_layer)

figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
#weather conditions - temperature lower than 80 degrees but higher than 70. Wind speed less than 10 mph. Zero cloudiness.

city_temperature = (city_data.Temperature < 80) & (city_data.Temperature > 70)
city_wind = city_data.Wind < 10 
city_clouds = city_data.Cloudiness == 0
final_conditions = city_temperature & city_wind & city_clouds

#filter through cities for above conditions & drop null 
best_weather = city_data[final_conditions]
best_weather = best_weather.dropna()
best_weather = best_weather.reset_index()

best_weather.head(10)

,index,Unnamed: 0,City,Latitude,Longitude,Temperature,Cloudiness,Humidity,Wind,Country,Date
0,107,107,Salalah,17.0151,54.0924,78.80,0,73,3.44,OM,1616264568
1,213,213,Dawei,14.0833,98.2000,74.48,0,80,2.57,MM,1616264586
2,258,258,Dhubri,26.0333,89.9667,74.98,0,32,4.09,IN,1616264631
3,264,264,Umluj,25.0213,37.2685,79.39,0,43,8.28,SA,1616264633
4,265,265,Māngrol,21.1167,70.1167,77.18,0,69,9.06,IN,1616264633
5,342,342,Bārmer,25.7500,71.3833,79.75,0,21,7.20,IN,1616264662
6,480,480,Riyadh,24.6877,46.7219,77.00,0,20,6.91,SA,1616264710


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = best_weather
hotel_df["Hotel Name"] = ""

hotel_df.head()

,index,Unnamed: 0,City,Latitude,Longitude,Temperature,Cloudiness,Humidity,Wind,Country,Date,Hotel Name
0,107,107,Salalah,17.0151,54.0924,78.80,0,73,3.44,OM,1616264568,
1,213,213,Dawei,14.0833,98.2000,74.48,0,80,2.57,MM,1616264586,
2,258,258,Dhubri,26.0333,89.9667,74.98,0,32,4.09,IN,1616264631,
3,264,264,Umluj,25.0213,37.2685,79.39,0,43,8.28,SA,1616264633,
4,265,265,Māngrol,21.1167,70.1167,77.18,0,69,9.06,IN,1616264633,


In [34]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#set parameters to search for hotels
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

In [52]:
#loop through data for hotels
for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{latitude},{longitude}"
    
    #make API request
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #save hotel to dataframe
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("No Hotel Found")
        
    print("Hotel Found")
    
print("-- Search Complete --")

Hotel Found
Hotel Found
Hotel Found
Hotel Found
No Hotel Found
Hotel Found
Hotel Found
Hotel Found
-- Search Complete --


In [50]:
hotel_df

,index,Unnamed: 0,City,Latitude,Longitude,Temperature,Cloudiness,Humidity,Wind,Country,Date,Hotel Name
0,107,107,Salalah,17.0151,54.0924,78.80,0,73,3.44,OM,1616264568,Star Emirates garden villas
1,213,213,Dawei,14.0833,98.2000,74.48,0,80,2.57,MM,1616264586,The Mandolis Hotel and Restaurant
2,258,258,Dhubri,26.0333,89.9667,74.98,0,32,4.09,IN,1616264631,SPOT ON 47805 The Hotel Brahmaputra
3,264,264,Umluj,25.0213,37.2685,79.39,0,43,8.28,SA,1616264633,HP Red Sea Hotel
4,265,265,Māngrol,21.1167,70.1167,77.18,0,69,9.06,IN,1616264633,
5,342,342,Bārmer,25.7500,71.3833,79.75,0,21,7.20,IN,1616264662,Sanchal Fort
6,480,480,Riyadh,24.6877,46.7219,77.00,0,20,6.91,SA,1616264710,Centro Olaya


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [57]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content=[info_box_template.format(**row) 
                                                               for index, row in hotel_df.iterrows()])

# Display figure
figure.add_layer(hotel_marker)

figure

Figure(layout=FigureLayout(height='420px'))